In [25]:
import requests
import pandas as pd
from selenium import webdriver
from scrapy.http import TextResponse
import seaborn as sns
import getpass
import time


###

In [26]:
# 여기서부터 시작

In [ ]:
login_url = "https://www.instagram.com/accounts/login/?source=auth_switcher"

driver = webdriver.Chrome()
driver.get(login_url)

In [88]:
# 로그인 함수 (seleniun webdriver)

def login():
#     login_url = "https://www.instagram.com/accounts/login/?source=auth_switcher"

#     driver = webdriver.Chrome()
#     driver.get(login_url)
    
    my_id = input('id를 입력하세요: ')
    my_password = getpass.getpass('password를 입력하세요: ')

    driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=text]').send_keys(my_id)
    driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=password]').send_keys(my_password)
    driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF[type=submit]').click()
    
    time.sleep(2.5)
    
    # 알림설정 하라는 modal창이 뜨는경우 '나중에 하기'를 클릭하는 코드
    alert_modal = driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div')
    if alert_modal:
        driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div.mt3GC > button.aOOlW.HoLwm').click()
    
    
    

In [89]:
# 검색어를 입력하고 게시물이 가장 많은 검색결과를 클릭 (seleniun webdriver)

def crawling_keyword(word):
    keyword = word

    driver.find_element_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > input').send_keys(keyword)

    # 키워드 입력하고, 키워드에 대한 검색 리스트가 뜨는데 걸리는 시간만큼 기다려준다
    time.sleep(1.5)
    
    search_list = driver.find_elements_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > div:nth-child(4) > div.drKGC > div > a')
    
    # 서칭된 a 엘리먼트들 중에 게시물 갯수가 가장 많은 a 엘리먼트를 뽑는다
    number_list = []

    for element in search_list:
        splited_element = element.text.split('게시물')
        if len(splited_element) == 2:
            number_list.append(int(re.sub(",", "", splited_element[1])))
        else:
            number_list.append(0)

    max_number = max(number_list)
    click_index = number_list.index(max_number)
    
    # 클릭하고 렌더링이 완료되도록 조금 기다리기
    search_list[click_index].click()
    time.sleep(1)


In [90]:
# 인기게시물 9개는 빼고, 최근사진부터 크롤링 시작(인기게시물과 최근게시물에 중복 게시물이 있을거 같아서)

In [91]:
# 처음 렌더링된 화면의 24개의 게시물을 크롤링하는 함수

def initial_crawling():
    urls = driver.find_elements_by_xpath(
        '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
    urls = [url.get_attribute("href") for url in urls]

    hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

    for i in range(len(urls)):
        req = requests.get(urls[i])
        response = TextResponse(req.url, body=req.text, encoding="utf-8")
        hash_tag_list = response.css("meta[property='instapp:hashtags']")
        hash_tag_list = ['#' + hash_tag.attrib['content']
                         for hash_tag in hash_tag_list]
        hash_tags = ', '.join(hash_tag_list)

        data = [
            {'time': 0, 'hash_tag': hash_tags}
        ]
        df = pd.DataFrame(data)
        hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [104]:
# 스크롤내리면서 12개씩 추가되는 게시물을 크롤링하는 함수 (두번째만 실행) - chrome 전체화면 기준
# 현재 브라우저 화면 크기에 따라 추가적으로 불러오는 div 갯수가 달라짐
# 전체화면 기준 div의 urls 총 갯수 = 24 -> 36 -> 45 -> 45 -> ...

def second_crawling():
    
    hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])
    
    for i in range(0, 1):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(1)

        urls = driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        for i in range(len(urls) - 12, len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content']
                             for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [105]:
# 세번째부터는 이 함수 반복하기 -  게시물 9개씩 크롤링

def repeat_crawling(num):
    
    hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])
    
    for i in range(0, num):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(1)

        urls = driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        for i in range(len(urls) - 9, len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content']
                             for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [94]:
# %%writefile insta_crawling.py
# import requests
# import pandas as pd
# from selenium import webdriver
# from scrapy.http import TextResponse
# import seaborn as sns
# import getpass
# import time


class instagram_crawling():
    
    driver = ""
    
    def __init__(self):
        login_url = "https://www.instagram.com/accounts/login/?source=auth_switcher"
        self.driver = webdriver.Chrome()
        self.driver.get(login_url)
    
    
    # 로그인 함수 (seleniun webdriver)
    def login(self):

        my_id = input('id를 입력하세요: ')
        my_password = getpass.getpass('password를 입력하세요: ')

        self.driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=text]').send_keys(my_id)
        self.driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=password]').send_keys(my_password)
        self.driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF[type=submit]').click()

        time.sleep(3)

        # 알림설정 하라는 modal창이 뜨는경우 '나중에 하기'를 클릭하는 코드
        alert_modal = self.driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div')
        if alert_modal:
            self.driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div.mt3GC > button.aOOlW.HoLwm').click()




    # 검색어를 입력하고 게시물이 가장 많은 검색결과를 클릭 (seleniun webdriver)
    def crawling_keyword(self, word):
        keyword = word

        self.driver.find_element_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > input').send_keys(keyword)

        # 키워드 입력하고, 키워드에 대한 검색 리스트가 뜨는데 걸리는 시간만큼 기다려준다
        time.sleep(1.5)

        search_list = self.driver.find_elements_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > div:nth-child(4) > div.drKGC > div > a')

        # 서칭된 a 엘리먼트들 중에 게시물 갯수가 가장 많은 a 엘리먼트를 뽑는다
        number_list = []

        for element in search_list:
            splited_element = element.text.split('게시물')
            if len(splited_element) == 2:
                number_list.append(int(re.sub(",", "", splited_element[1])))
            else:
                number_list.append(0)

        max_number = max(number_list)
        click_index = number_list.index(max_number)

        # 클릭하고 렌더링이 완료되도록 조금 기다리기
        search_list[click_index].click()
        time.sleep(1.5)




    # 처음 렌더링된 화면의 24개의 게시물을 크롤링하는 함수
    def initial_crawling(self):
        urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

        for i in range(len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

        return hash_tag_df.reset_index().drop(columns=['index'])




    # 스크롤내리면서 12개씩 추가되는 게시물을 크롤링하는 함수 (두번째만 실행) - chrome 전체화면 기준
    # 현재 브라우저 화면 크기에 따라 추가적으로 불러오는 div 갯수가 달라짐
    # 전체화면 기준 div의 urls 총 갯수 = 24 -> 36 -> 45 -> 45 -> ...
    def second_crawling(self):
        
        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])
        
        for i in range(0, 1):
            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(1.5)

            urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
            urls = [url.get_attribute("href") for url in urls]

            for i in range(len(urls) - 12, len(urls)):
                req = requests.get(urls[i])
                response = TextResponse(req.url, body=req.text, encoding="utf-8")
                hash_tag_list = response.css("meta[property='instapp:hashtags']")
                hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
                hash_tags = ', '.join(hash_tag_list)

                data = [
                    {'time': 0, 'hash_tag': hash_tags}
                ]
                df = pd.DataFrame(data)
                hash_tag_df = hash_tag_df.append(df)

        return hash_tag_df.reset_index().drop(columns=['index'])



    # 세번째부터는 이 함수 반복하기 - 게시물 9개씩 크롤링
    def repeat_crawling(self, num):
        
        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])
        
        for i in range(0, num):
            try:
                self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                time.sleep(1.5)

                urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
                urls = [url.get_attribute("href") for url in urls]

                for i in range(len(urls) - 9, len(urls)):
                    req = requests.get(urls[i])
                    response = TextResponse(req.url, body=req.text, encoding="utf-8")
                    hash_tag_list = response.css("meta[property='instapp:hashtags']")
                    hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
                    hash_tags = ', '.join(hash_tag_list)

                    data = [
                        {'time': 0, 'hash_tag': hash_tags}
                    ]
                    df = pd.DataFrame(data)
                    hash_tag_df = hash_tag_df.append(df)
                    
            except Exception as exc:
                print('에러가 발생했습니다 : ', exc)

        return hash_tag_df.reset_index().drop(columns=['index'])


In [95]:
insta = instagram_crawling()

In [96]:
insta.login()

id를 입력하세요: yeejun90
password를 입력하세요: ········


In [97]:
insta.crawling_keyword('초코우유')

In [98]:
insta.initial_crawling()

,time,hash_tag
0,0,"#달콤, #소풍스타그램, #티라미수, #핵잼, #초코우유, #홍차, #홍대카페, #..."
1,0,"#소통, #카페투어, #답례품, #코로나저리가, #수제간식, #행사간식, #구움과자..."
2,0,"#망고우유, #산딸기요거트, #얼그레이, #오레오, #인절미, #라임, #초코우유,..."
3,0,"#푸들, #dogstagram, #비숑프리제, #dogs, #비숑스타그램, #심장뿌..."
4,0,"#카페, #ootd, #일상, #인친, #좋반, #like, #팔로워, #데일리, ..."
5,0,"#감성, #dm, #선팔하면맞팔, #셀카, #초코우유, #분위기, #네일매니저, #..."
6,0,"#펭수, #ebs, #펭카, #자이언트펭티비, #펭럽유, #giantpengtv, ..."
7,0,#초코우유
8,0,"#초코바라기, #초코아이스크림, #초코요정, #초코우유"
9,0,"#초코우유, #김치, #짜파게티"


In [99]:
insta.second_crawling()

,time,hash_tag
0,0,"#저녁식사, #유어스, #참깨라면, #지에스25, #펭럽유초코, #음식스타그램, #..."
1,0,"#팔로우, #까르보나라, #선팔, #likeforlikeback, #좋반, #맞팔해..."
2,0,"#여신파마, #가천여행, #선플, #세종카페, #스위스여행, #뮤비촬영, #초코우유..."
3,0,"#일상, #일상그램, #f4, #댓글, #오늘의간식, #선팔맞팔, #소통환영, #일..."
4,0,
5,0,"#생일, #탄생화컵, #직접만든, #홈카페놀이, #🍫, #홈카페, #초코우유, #생..."
6,0,"#꼭, #초코우유, #어피치덕후, #사탕은, #그게, #행복하다그램, #어피치는, ..."
7,0,"#펭럽유초코, #펭럽유초코우유, #펭수, #초코우유, #gs25"
8,0,
9,0,"#마카롱꼬끄, #스믈여덟마카롱, #초코우유, #리얼쇼콜라"


In [100]:
insta.repeat_crawling(3)

,time,hash_tag
0,0,
1,0,"#사장님과토크ㅋ, #역시, #코스가, #인, #초코우유, #옴마, #저번에, #레몬..."
2,0,"#카페, #한옥, #제주리얼말차라떼, #맛있다, #다음기회에👋, #소금앙버터, #미..."
3,0,"#초코우유, #쌍문동마카롱, #달달, #일상, #선물, #macarons, #코로나..."
4,0,"#방학중, #도너츠, #초코우유, #아들둘맘, #좋아하는것도다다름, #아들맘, #바..."
5,0,"#macaron, #오레오, #화이트데이, #충장로, #까페모카, #말차가나슈, #..."
6,0,"#초코우유, #김남길생각, #앤드, #칼로리바란스"
7,0,"#수제우유, #망원동, #까눌레맛집, #단호박우유, #보틀우유, #카페망원정, #쑥..."
8,0,"#초코우유, #아들몰래먹기, #미안해아들, #꿀맛💕, #엄마스트레스해소용"
9,0,


In [21]:
import pprint

In [17]:
# 맨처음 화면의 게시물 갯수

urls = driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
urls = [url.get_attribute("href") for url in urls]
len(urls)

24

In [577]:
# 스크롤을 한번 끝까지 내렸을때 새로 불러온 게시물을 포함한 전체 게시물 갯수

driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
time.sleep(1.5)
urls = driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
urls = [url.get_attribute("href") for url in urls]
len(urls)

45

In [18]:
req = requests.get(urls[0])
req.url

'https://www.instagram.com/p/B9pxuKenKyG/'

In [19]:
response = TextResponse(req.url, body=req.text, encoding="utf-8")

response

<200 https://www.instagram.com/p/B9pxuKenKyG/>

In [78]:
# pprint.pprint(response.text)

In [581]:
response.xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[2]/div[1]/ul/div/li/div/div/div[2]/span/a')

[]

In [79]:
# pprint.pprint(req.text)

In [32]:
driver.quit()